# TRABAJO FINAL - APRENDIZAJE AUTOMÁTICO
###  - Nuria Bango Iglesias
###  - Álvar De Diego López

# Abalone UCI
### Prediciendo la edad de los abalones dadas medidas físicas. 

***

## Contexto

El abulón es el nombre común de un grupo de caracoles marinos de pequeño a gran tamaño, que se encuentran habitualmente en las costas de todo el mundo y se utilizan como manjar en las cocinas. Su concha sobrante se utiliza en joyería debido a su brillo iridiscente. 

Debido a su demanda y valor económico, a menudo se recolecta en granjas, por lo que es necesario predecir la edad del abalón a partir de medidas físicas. El método tradicional para determinar su edad consiste en cortar la concha a través del cono, teñirla y contar el número de anillos a través de un microscopio, una tarea aburrida y que requiere mucho tiempo, por lo que predecir la edad utilizando otros factores como el peso o la altura que se pueden medir fácilmente agilizaría el proceso.

Sabemos que a medida que el abalón crece añade anillos, y a medida que añade nuevas capas a la concha aumenta el peso y el diámetro de la misma. Se puede ver en la imagen de abajo:


<img src="abalone_rings.jpg" width="500" />

***

## Datos

Número de instancias: 4177

Número de atributos: 8

Objetivo: Anillos (Rings)

<img src="abalone_partes.jpg" width="500" />

| Atributo | Tipo de datos| Unidades | Descripción |
| :----: | :----: | :----: | :----: |
| Sex | nominal | - | M (masculino), F (femenino), I (infantes) | 
| Length | continuous | mm |  Longitud de la cáscara | 
| Diameter | continuous | mm | Perpendicular a la longitud | 
| Height | continuous | mm | Con carne en la cáscara | 
| Whole weight | continuous | grams | Abalón completo | 
| Shucked weight | continuous | grams | Peso de la carne |
| Viscera weight | continuous | grams | Peso de la tripa (después del sangrado) |
| Shell weight | continuous | grams | Después de ser secado |
| Rings | integer | - | +1.5 da la edad en años | 



## Primeros pasos

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
import matplotlib.pyplot as plt # plotting
import scipy.stats as stats # Stats API
import seaborn as sns 
sns.set(style='whitegrid', palette='colorblind') 

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import svm
from sklearn.svm import SVR
from sklearn.svm import SVC
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('abalone_original.csv')
df.dataframeName = 'abalone_original.csv'

In [ ]:
df

In [ ]:
df.info()

# Análisis de variables individuales

***

## Sexo

A partir de los campos de descripción de los datos iniciales sabemos que el sexo se clasifica en Macho, Hembra, lo cual tiene sentido, mientras que Infante parece estar mal clasificado o fue clasificado así ya que era difícil decir el sexo en el momento de la observación ya que el Abalón era Infante.

Vamos a confirmar primero que son 3 las categorias y cual de ellas es más frecuente.

In [ ]:
df.sex.describe()

Podemos confirmar que la categoría sexo sólo tiene 3 campos, principalmente: masculino, femenino e infantil, siendo el masculino el más frecuente. 

Veamos su densidad, para entender en qué medida es mayor el sexo masculino que el femenino y el infantil en este conjunto de datos.

In [ ]:
df.sex.value_counts(normalize=True).sort_index()

Por lo tanto, hay un número significativamente mayor de hombres que de otras dos categorías, visualicemos ya que las mujeres y los niños están muy cerca utilizando la densidad relativa para tener una idea de sus diferencias.

In [ ]:
df_sex_category = df.sex.value_counts(normalize=True).sort_index()
x = range(len(df_sex_category))
figure = plt.figure(figsize=(10, 6))
axes1 = figure.add_subplot(1, 2, 1)
axes1.bar(x, df_sex_category, color="steelblue",align="center")
axes1.set_xticks(x)
# Set x axis tick labels
axes1.set_xticklabels(df_sex_category.axes[0])
# Set x and y axis chart label
axes1.set_title("Sex Categories")
axes1.set_ylabel("Density")
axes1.xaxis.grid(False)
# Remove all of the axis tick marks
axes1.tick_params(bottom=False, top=False, left=False, right=False)
# Hide all of the spines
for spine in axes1.spines.values():
    spine.set_visible(False)
axes1.yaxis.grid(b=True, which="major")

Hay aproximadamente la misma proporción de mujeres y niños, con un poco más de hombres que de mujeres o niños.

Una cosa que podemos notar, es que presumiblemente el sexo es mucho más difícil de reconocer en los infantes y esto es evidente basado en que tenemos ligeramente más infantes que hembras.

## Longitud

Hagamos una estadística de resumen para ver cómo se distribuye la longitud, esperamos que siga una distribución normal, con algo de asimetría teniendo en cuenta que tenemos niños en el conjunto de datos.

In [ ]:
df.length.describe()

Lo primero que destaca es que el mínimo es realmente pequeño, 15 mm, lo que tiene sentido debido a los bebés en el conjunto de datos, que puede ser la razón por la que la distribución tendría una distribución sesgada a la izquierda.

Vamos a visualizarlo con un gráfico de caja para verificarlo, y ver si hay alguna anomalía como valores atípicos en la longitud.


In [ ]:
def restyle_boxplot(patch):
    # change color and linewidth of the whiskers
    for whisker in patch['whiskers']:
        whisker.set(color='#000000', linewidth=1)

    # change color and linewidth of the caps
    for cap in patch['caps']:
        cap.set(color='#000000', linewidth=1)

    # change color and linewidth of the medians
    for median in patch['medians']:
        median.set(color='#000000', linewidth=2)

    # change the style of fliers and their fill
    for flier in patch['fliers']:
        flier.set(marker='o', color='#000000', alpha=0.2)

    for box in patch["boxes"]:
        box.set(facecolor='#FFFFFF', alpha=0.5)

def numeric_boxplot(numeric_df, label, title):
    figure = plt.figure(figsize=(20, 6))
    figure.suptitle(title)
    axes1 = figure.add_subplot(1, 2, 1)
    patch = axes1.boxplot(numeric_df, labels=[label], vert=False, showfliers = True, patch_artist=True, zorder=1)
    restyle_boxplot(patch)
    axes1.set_title('Diagrama de cajas 1')
    axes2 = figure.add_subplot(1, 2, 2)
    patch = axes2.boxplot(numeric_df, labels=[label], vert=False, patch_artist=True, zorder=1)
    restyle_boxplot(patch)
    axes2.set_title('Diagrama de cajas 2')
    y = np.random.normal(1, 0.01, size=len(numeric_df))
    axes2.plot(numeric_df, y, 'o', color='steelblue', alpha=0.4, zorder=2)
    plt.show()
    plt.close()

numeric_boxplot(df.length, 'Length', 'Distribución de la longitud')

El diagrama 1 nos dice que existen valores atípicos entre los rangos de 20 a 40, lo que presumiblemente se debe a la presencia de infantes o de especies particulares de abulón de pequeño tamaño.

En el diagrama 2 podemos ver que la distribución está efectivamente sesgada a la izquierda y que los valores se centran en el rango 100-140.

## Peso

De las cuatro medidas de peso diferentes: El peso de la cáscara, el peso de las vísceras, el peso del descascarillado y alguna masa desconocida de agua/sangre perdida por el proceso de descascarillado, intentaremos sacar conclusiones como peso total.

Peso total = peso de la cáscara+peso de la víscera+peso del descascarillado+masa desconocida de agua/sangre perdida

Analizaremos todos los pesos juntos ya que están relacionados entre sí y deberían tener estadísticas similares.

Exploremos primero las estadísticas para todas las características del peso:


In [ ]:
df[['whole-weight', 'shucked-weight', 'viscera-weight', 'shell-weight']].describe()

De las estadísticas resumidas anteriores podemos ver que cada característica de peso tiene una media mayor que la mediana, por lo que podemos esperar una asimetría derecha en la distribución, a diferencia de la longitud y el diámetro.

Para todo el peso, podemos ver que los rangos de peso van desde el más bajo de alrededor de 0 gramos hasta el máximo que pesa alrededor de 570 gramos.

Es de esperar una tendencia similar, y parece que los pesos tienen una alta varianza, y esto vuelve a lo que dijimos antes, en cuanto a que el abalón es una colección de tamaños pequeños y grandes, por lo que deberíamos esperar que los pesos varíen entre los abalones.

Ahora vamos a visualizar la dispersión con un histograma de densidad para ver si parece normal y tiene la asimetría correcta.


In [ ]:
def freeman_diaconis(data):
    quartiles = stats.mstats.mquantiles(data, [0.25, 0.5, 0.75])
    iqr = quartiles[2] - quartiles[0]
    n = len(data)
    h = 2.0 * (iqr/n**(1.0/3.0))
    return int(h)

weights = ['whole-weight', 'shucked-weight', 'viscera-weight', 'shell-weight']
figure = plt.figure(figsize=(20, 10))
#figure = plt.subplots(nrows=2, ncols=2)
for i, k in enumerate(weights):
    axes = figure.add_subplot(2, 2, i + 1)
    subdata = df[k]
    binwidth = freeman_diaconis(subdata)
    bins = np.arange(min(subdata), max(subdata) + binwidth, binwidth)
    axes.hist(subdata, color="steelblue", bins=bins, density=True, alpha=0.75)
    axes.xaxis.grid(False)
    axes.set_title("Densidad de {}".format(k))
    if (i % 3 == 0):
        axes.set_ylabel("Density")
    axes.set_xlabel(k)
plt.tight_layout()


De hecho, la distribución está sesgada a la derecha y el peso de la cáscara y el peso de las vísceras son similares al peso entero y entre sí.

Ahora podemos esperar que el peso entero esté altamente correlacionado con otras variables de peso, lo que no nos ayuda mucho a predecir la edad.

## Anillos

Sabemos que los anillos son un predictor de la edad, por lo que son valores enteros, ya que se cuentan como una cantidad discreta en lugar de valores continuos, por lo que deberíamos ver distribuciones discretas con los anillos, y los recuentos serían más apropiados para el eje Y.

Ahora vamos a empezar con las estadísticas de resumen para ver cuál es el número mínimo y máximo de anillos, y dónde se encuentra la media de la muestra.


In [ ]:
df.rings.describe()


Podemos ver que la mayoría de los anillos están entre 8 y 10, mientras que la media está en torno a 10, y como la media es mayor que la mediana, la distribución sería sesgada a la derecha. La varianza de la muestra es menor, por lo que la distribución sería más estrecha, podemos trazar un histograma de densidad para comprobarlo.


In [ ]:
bins = np.arange(min(df.rings), max(df.rings) + binwidth, binwidth)
figure = plt.figure(figsize=(10, 6))
axes = figure.add_subplot(1, 1, 1)
axes.hist(df.rings, bins=20, density=True, alpha=0.75)
axes.set_ylabel("Density")
axes.set_xlabel("Rings")
axes.set_title("Histograma de densidad de los anillos")
axes.xaxis.grid(False)
plt.show()

Efectivamente, es como esperábamos, la distribución es sesgada a la derecha y tiene menos dispersión, con una media en torno a 10, por lo que la edad del abalón estaría en torno a los 11,5 años.

# Análasis a pares

Antes de realizar el análisis por pares, visualicemos la matriz de coeficientes de correlación en forma de mapa de calor para entender qué análisis por pares debemos comprobar.

In [ ]:
corr = df.corr('spearman')
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
f, ax = plt.subplots(figsize=(8, 6))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(corr, mask=mask, vmax=1,square=True, 
            linewidths=.5, cbar_kws={"shrink": .5}, ax=ax, annot=True)
plt.show()
plt.close()

Observamos que muchas de las variables están altamente correlacionadas entre sí con r > 0,7, podemos esperar que los pesos estén relacionados entre sí como vimos en el análisis univariante, pero es inesperado ver que la altura también está relacionada con los pesos, y la longitud y el diámetro.

Podemos ignorar las otras variables de peso ya que no nos dan ninguna información nueva que conozcamos, también esperamos saber que la longitud, la altura y el diámetro están relacionados linealmente, para el cálculo de peso habría que centrarse en:

- la altura frente al peso total
- la longitud frente al peso total
- el diámetro frente al peso total.

Con estas conclusiones podemos aventurarnos a decir que hay una correlación algo débil entre los anillos y otras variables.


# Preprocesamiento de datos

In [ ]:
def preprocess_and_train(df, target, task):
    df = df.copy()
    
    if target != 'sex':
        dummies = pd.get_dummies(df['sex'])
        df = pd.concat([df, dummies], axis=1)
        df = df.drop('sex', axis=1)
    
    y = df[target].copy()
    X = df.drop(target, axis=1).copy()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)
    
    scaler = StandardScaler()
    scaler.fit(X_train)
    X_train = pd.DataFrame(scaler.transform(X_train),columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    
    if task == 'linear':
        model = LinearRegression()
    elif task == 'logistic':
        model = LogisticRegression()
    elif task == 'svm':
        model = SVC(random_state = 1)
    
    model.fit(X_train, y_train)

    return model.score(X_test, y_test)

## Prediciendo Sexo

In [ ]:
results = preprocess_and_train(df, target='sex', task='logistic')

print("Precisión de la regresión logistica en Sex: {:.2f}%".format(results * 100))

In [ ]:
results = preprocess_and_train(df, target='sex', task='svm')

print("Precisión de SVM en Sex: {:.2f}%".format(results * 100))

## Prediciendo Longitud

In [ ]:
results = preprocess_and_train(df, target='length', task='linear')

print("Precisión de la regresión lineal en length: {:.4f}%".format(results * 100))

## Prediciendo Diámetro

In [ ]:
results = preprocess_and_train(df, target='diameter', task='linear')

print("Precisión de la regresión lineal en diameter: {:.4f}%".format(results * 100))

## Prediciendo Altura

In [ ]:
results = preprocess_and_train(df, target='height', task='linear')

print("Precisión de la regresión lineal en height: {:.4f}%".format(results * 100))

## Prediciendo "Whole-Weight"


In [ ]:
results = preprocess_and_train(df, target='whole-weight', task='linear')

print("Precisión de la regresión lineal en whole-weight: {:.4f}%".format(results * 100))

## Prediciendo "Shucked-Weight"

In [ ]:
results = preprocess_and_train(df, target='shucked-weight', task='linear')

print("Precisión de la regresión lineal en shucked-weight: {:.4f}%".format(results * 100))

## Prediciendo "Viscera-Weight"


In [ ]:
results = preprocess_and_train(df, target='viscera-weight', task='linear')

print("Precisión de la regresión lineal en viscera-weight: {:.4f}%".format(results * 100))

## Prediciendo "Shell-Weight"

In [ ]:
results = preprocess_and_train(df, target='shell-weight', task='linear')

print("Precisión de la regresión lineal en shell-weight: {:.4f}%".format(results * 100))

## Prediciendo Anillos


In [ ]:
results = preprocess_and_train(df, target='rings', task='linear')

print("Precisión de la regresión lineal en rings: {:.4f}%".format(results * 100))

In [ ]:
results = preprocess_and_train(df, target='rings', task='logistic')

print("Precisión de la regresión logistica en rings: {:.4f}%".format(results * 100))

# Bibliografía

***

El link principal de referencia de Kaggle es: https://www.kaggle.com/hurshd0/abalone-uci

El Dataset fue sacado originalmente del repositorio de UCI Machine Learning: https://archive.ics.uci.edu/ml/datasets/Abalone
